# TSA Chapter 3: Variance Growth: RW vs Stationary

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_variance_growth/TSA_ch3_variance_growth.ipynb)

This notebook demonstrates:
- Variance comparison: random walk Var grows linearly with time vs stationary AR(1) bounded variance sigma^2/(1-phi^2).


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
n = 500
n_sims = 2000

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Random walk variance
rw_paths = np.cumsum(np.random.normal(0, 1, (n_sims, n)), axis=1)
rw_var = np.var(rw_paths, axis=0)
axes[0].plot(rw_var, color='#DC3545', linewidth=2, label='Empirical')
axes[0].plot(np.arange(1, n+1), color='#1A3A6E', linewidth=2, linestyle='--', label=r'$t \cdot \sigma^2$')
axes[0].set_title('Random Walk: Unbounded Variance', fontweight='bold')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Variance')
axes[0].legend()

# Stationary AR(1) variance
phi = 0.8
ar_paths = np.zeros((n_sims, n))
for sim in range(n_sims):
    eps = np.random.normal(0, 1, n)
    for t in range(1, n):
        ar_paths[sim, t] = phi * ar_paths[sim, t-1] + eps[t]

ar_var = np.var(ar_paths, axis=0)
theory_var = 1 / (1 - phi**2)
axes[1].plot(ar_var, color='#2E7D32', linewidth=2, label='Empirical')
axes[1].axhline(y=theory_var, color='#1A3A6E', linewidth=2, linestyle='--',
                label=f'$\\sigma^2/(1-\\phi^2) = {theory_var:.2f}$')
axes[1].set_title(f'AR(1) $\\phi={phi}$: Bounded Variance', fontweight='bold')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Variance')
axes[1].legend()
axes[1].set_ylim(0, theory_var * 2)

plt.tight_layout()
save_chart(fig, 'ch3_variance_growth')
plt.show()